In [69]:
import os
import numpy as np
import nibabel as nib
import torch
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from ipywidgets import interact, IntSlider

def load_image(file_path):
    if file_path.endswith('.nii') or file_path.endswith('.nii.gz'):
        try:
            img = nib.load(file_path)
            data = img.get_fdata()
            if data.ndim == 4 and data.shape[0] == 1:
                data = data[0]  # Remove leading singleton dimension
            return data
        except Exception as e:
            print(f"Error loading NIfTI file {file_path}: {e}")
            return None
    elif file_path.endswith('.pth'):
        try:
            data = torch.load(file_path)
            if isinstance(data, torch.Tensor):
                data = data.numpy()
            if data.ndim == 4 and data.shape[0] == 1:
                data = data[0]
            return data
        except Exception as e:
            print(f"Error loading PyTorch file {file_path}: {e}")
            return None
    else:
        print(f"Unsupported file format for: {file_path}")
        return None

def visualize_slices(original_path, prediction_path, overlay_style='colormap'):
    data_original = load_image(original_path)
    data_prediction = load_image(prediction_path)

    if data_original is None or data_prediction is None:
        print("Cannot visualize: one or both files failed to load.")
        return

    def plot_comparison_slices(slice_index):
        fig, axes = plt.subplots(1, 2, figsize=(16, 8))
        
        original_slice = data_original[:, :, slice_index]
        prediction_slice = data_prediction[:, :, slice_index]

        original_rotated = np.rot90(original_slice, k=3)
        prediction_rotated = np.rot90(prediction_slice, k=3)

        axes[0].imshow(original_rotated, cmap='gray')
        axes[0].set_title(f'Original Image (Slice {slice_index}, Rotated)')
        axes[0].axis('off')

        axes[1].imshow(original_rotated, cmap='gray')

        prediction_mask = (prediction_rotated == 0)

        if overlay_style == 'colormap':
            axes[1].imshow(np.ma.masked_where(prediction_mask, prediction_rotated),
                           cmap='viridis', alpha=0.5, vmin=1, vmax=9)
        elif overlay_style == 'yellow':
            colors = [(0, 0, 0, 0), (1, 1, 0, 1)]
            custom_cmap = ListedColormap(colors)
            axes[1].imshow(np.ma.masked_where(prediction_mask, prediction_rotated),
                           cmap=custom_cmap, alpha=0.5, vmin=0, vmax=1)
        else:
            raise ValueError("Unsupported overlay style")

        axes[1].set_title(f'Prediction Overlay (Slice {slice_index}, Rotated)')
        axes[1].axis('off')

        plt.tight_layout()
        plt.show()

    max_slice = min(data_original.shape[2], data_prediction.shape[2]) - 1
    interact(plot_comparison_slices, slice_index=IntSlider(min=0, max=max_slice, step=1, value=max_slice // 2))


In [ ]:
# Example usage
visualize_slices(
    original_path='/home/ffati/UnderXAI-OVIT/CT.nii',
    prediction_path='/home/ffati/UnderXAI-OVIT/ovseg_predictions_pod_om/CT.nii',
    overlay_style='yellow'  


interactive(children=(IntSlider(value=85, description='slice_index', max=170), Output()), _dom_classes=('widge…

In [73]:
# Example usage
visualize_slices(
    original_path='/home/ffati/UnderXAI-OVIT/processed_data/pth/CT.pth',
    prediction_path='/home/ffati/UnderXAI-OVIT/processed_data/pth/OC_mask.pth',
    overlay_style='yellow'  # or 'colormap'
)


You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.


interactive(children=(IntSlider(value=127, description='slice_index', max=255), Output()), _dom_classes=('widg…